In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('./data.csv')

In [3]:
X,y = data.drop('target',axis=1),data['target']

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25)

In [6]:
import torch
import torch.nn as nn

In [7]:
import numpy as np

In [8]:
X_train = torch.from_numpy(np.array(X_train).astype(np.float32))
y_train = torch.from_numpy(np.array(y_train).astype(np.float32))
X_test = torch.from_numpy(np.array(X_test).astype(np.float32))
y_test = torch.from_numpy(np.array(y_test).astype(np.float32))

In [9]:
X_train.shape

torch.Size([227, 13])

In [10]:
X_test.shape

torch.Size([76, 13])

In [11]:
y_train.shape

torch.Size([227])

In [12]:
y_test.shape

torch.Size([76])

In [13]:
import torch.nn.functional as F

In [14]:
class Test_Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(13,64)
        self.fc2 = nn.Linear(64,128)
        self.fc3 = nn.Linear(128,256)
        self.fc4 = nn.Linear(256,512)
        self.fc5 = nn.Linear(512,1024)
        self.fc6 = nn.Linear(1024,512)
        self.fc7 = nn.Linear(512,1)
    
    def forward(self,X):
        preds = self.fc1(X)
        preds = F.relu(preds)
        preds = self.fc2(preds)
        preds = F.relu(preds)
        preds = self.fc3(preds)
        preds = F.relu(preds)
        preds = self.fc4(preds)
        preds = F.relu(preds)
        preds = self.fc5(preds)
        preds = F.relu(preds)
        preds = self.fc6(preds)
        preds = F.relu(preds)
        preds = self.fc7(preds)
        return F.sigmoid(preds)

In [15]:
device = torch.device('cuda')

In [16]:
X_train = X_train.to(device)
y_train = y_train.to(device)
X_test = X_test.to(device)
y_test = y_test.to(device)

In [17]:
PROJECT_NAME = 'Heart-Disease-UCI'

In [18]:
def get_loss(criterion,X,y,model):
    model.eval()
    with torch.no_grad():
        preds = model(X.float().to(device))
        preds = preds.to(device)
        y = y.to(device)
        loss = criterion(preds,y)
    model.train()
    return loss.item()
def get_accuracy(X,y,model):
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for i in range(len(X)):
            pred = model(X[i].float().to(device))
            pred.to(device)
            if round(int(pred[0])) == round(int(y[i])):
                correct += 1
            total += 1
        if correct == 0:
            correct += 1
    model.train()
    return round(correct/total,3)

In [19]:
import wandb

In [20]:
from tqdm import tqdm

In [21]:
EPOCHS = 212
# EPOCHS = 100

In [22]:
model = Test_Model().to(device)
optimizer = torch.optim.SGD(model.parameters(),lr=0.25)
criterion = nn.L1Loss()
wandb.init(project=PROJECT_NAME,name='baseline')
for _ in tqdm(range(EPOCHS)):
    preds = model(X_train.float().to(device))
    preds = preds.view(len(preds),)
    preds.to(device)
    loss = criterion(preds,y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    wandb.log({'loss':loss.item(),'val_loss':get_loss(criterion,X_test,y_test,model),'accuracy':get_accuracy(X_train,y_train,model),'val_accuracy':get_accuracy(X_test,y_test,model)})
wandb.finish()

In [23]:
preds

tensor([9.9986e-01, 9.7741e-01, 9.6176e-01, 9.9925e-01, 9.9968e-01, 6.8069e-01,
        9.9938e-01, 9.2089e-01, 7.7668e-03, 9.9562e-01, 9.9324e-01, 9.8979e-01,
        9.9963e-01, 9.9804e-01, 9.3612e-01, 9.8636e-01, 9.9869e-01, 8.4439e-01,
        9.9747e-01, 2.9402e-02, 9.9984e-01, 9.9985e-01, 6.1239e-02, 9.9902e-01,
        9.9993e-01, 9.9992e-01, 9.9993e-01, 9.9876e-01, 5.7726e-03, 1.4600e-02,
        9.8323e-01, 9.0594e-04, 9.8228e-01, 9.7572e-01, 9.8070e-01, 9.9331e-01,
        9.9902e-01, 9.8496e-01, 9.9980e-01, 9.9933e-01, 9.9870e-01, 9.4216e-02,
        9.9950e-01, 9.9996e-01, 9.7816e-01, 9.9896e-01, 9.9967e-01, 9.8593e-01,
        9.9998e-01, 8.9938e-01, 9.9739e-01, 9.9053e-01, 9.9711e-01, 9.9983e-01,
        9.9957e-01, 9.9144e-01, 9.8978e-01, 9.5015e-01, 6.8357e-01, 9.8906e-01,
        9.9971e-01, 9.9995e-01, 9.9858e-01, 6.6400e-03, 9.9356e-01, 9.9992e-01,
        9.9967e-01, 9.7453e-01, 1.3994e-02, 9.9973e-01, 9.9901e-01, 9.9983e-01,
        9.9400e-01, 9.9824e-01, 1.6461e-

In [24]:
preds = torch.round(preds)

In [25]:
correct = 0
total = 0

In [26]:
for real,pred in zip(y_train,preds):
    if real == pred:
        correct += 1
    total += 1

In [27]:
round(correct/total,3)

0.63

In [28]:
preds[0]

tensor(1., device='cuda:0', grad_fn=<SelectBackward>)

In [29]:
preds[1]

tensor(1., device='cuda:0', grad_fn=<SelectBackward>)

In [30]:
preds

tensor([1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 0., 1., 1., 0., 1., 1., 1., 1., 1., 0., 0., 1., 0., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 0., 1., 1., 1.,
        1., 1., 0., 0., 1., 0., 0., 1., 1., 1., 1., 1., 0., 0., 1., 1., 1., 1.,
        1., 1., 1., 1., 0., 0., 1., 0., 1., 0., 0., 1., 0., 1., 1., 1., 1., 1.,
        0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 0., 1.,
        1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 1., 1., 1., 0.,
        1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0.,
        1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0., 0., 1., 0., 1.,
        1., 1., 1., 1., 0., 0., 1., 1., 1., 0., 1., 1., 0., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 

In [31]:
preds

tensor([1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 0., 1., 1., 0., 1., 1., 1., 1., 1., 0., 0., 1., 0., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 0., 1., 1., 1.,
        1., 1., 0., 0., 1., 0., 0., 1., 1., 1., 1., 1., 0., 0., 1., 1., 1., 1.,
        1., 1., 1., 1., 0., 0., 1., 0., 1., 0., 0., 1., 0., 1., 1., 1., 1., 1.,
        0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 0., 1.,
        1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 1., 1., 1., 0.,
        1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0.,
        1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0., 0., 1., 0., 1.,
        1., 1., 1., 1., 0., 0., 1., 1., 1., 0., 1., 1., 0., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 

In [32]:
preds = torch.round(preds)

In [33]:
correct = 0
total = 0

In [34]:
for real,pred in zip(y_train,preds):
    if real == pred:
        correct += 1
    total += 1

In [35]:
round(correct/total,3)

0.63

In [36]:
preds[:10]

tensor([1., 1., 1., 1., 1., 1., 1., 1., 0., 1.], device='cuda:0',
       grad_fn=<SliceBackward>)

In [37]:
## Testing Modelling

In [38]:
import torch
import torch.nn as nn

In [39]:
X_train.shape

torch.Size([227, 13])

In [40]:
len(pred)

In [41]:
len(preds)

227

In [42]:
y_train

tensor([1., 1., 0., 1., 1., 0., 1., 1., 0., 1., 0., 0., 1., 1., 0., 1., 0., 1.,
        1., 0., 1., 0., 0., 1., 1., 1., 1., 1., 0., 0., 0., 1., 0., 0., 0., 0.,
        1., 0., 0., 0., 1., 0., 0., 1., 1., 1., 0., 1., 1., 0., 1., 0., 0., 1.,
        1., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1.,
        0., 1., 0., 1., 1., 1., 1., 1., 0., 1., 1., 0., 0., 0., 1., 0., 1., 1.,
        0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 1., 0., 0., 1., 1., 1., 1., 1.,
        0., 1., 0., 0., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 0., 1., 0., 0.,
        1., 1., 1., 1., 1., 0., 0., 0., 1., 1., 1., 1., 1., 1., 0., 0., 0., 1.,
        1., 1., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
        0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 0., 0., 0., 1., 0.,
        1., 1., 1., 1., 0., 1., 1., 1., 1., 0., 1., 1., 1., 0., 0., 0., 0., 1.,
        1., 0., 1., 1., 0., 0., 1., 0., 1., 1., 1., 0., 1., 0., 1., 0., 1., 0.,
        1., 0., 0., 1., 1., 0., 0., 0., 

In [43]:
y_test

tensor([0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1., 1., 1., 0.,
        1., 0., 0., 1., 1., 1., 1., 1., 1., 0., 1., 0., 0., 0., 1., 0., 0., 1.,
        0., 0., 1., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 1., 1., 0.,
        1., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 1., 0., 1., 1., 0.,
        0., 1., 0., 0.], device='cuda:0')

In [44]:
y_test.shape

torch.Size([76])

In [45]:
class Test_Model(nn.Module):
    def __init__(self,num_of_layers,activation=F.relu,input_shape=13,fc1_output=32,fc2_output=64,fc3_output=128,fc4_output=256,output_shape=1:
        super().__init__()
        self.num_of_layers = num_of_layers
        self.activation = activation
        self.fc1 = nn.Linear(input_shape,fc1_output)
        self.fc2 = nn.Linear(fc1_output,fc2_output)
        self.fc3 = nn.Linear(fc2_output,fc3_output)
        self.fc4 = nn.Linear(fc3_output,fc4_output)
        self.fc5 = nn.Linear(fc4_output,fc3_output)
        self.fc6 = nn.Linear(fc3_output,fc3_output)
        self.fc7 = nn.Linear(fc3_output,output_shape)
    
    def forward(self,X,activation=False):
        preds = self.fc1(X)
        if activation:
                 preds = self.activation(preds)
        preds = self.fc2(preds)
        if activation:
                 preds = self.activation(preds)
        preds = self.fc3(preds)
        if activation:
                 preds = self.activation(preds)
        preds = self.fc4(preds)
        if activation:
                 preds = self.activation(preds)
        preds = self.fc5(preds)
        if activation:
                 preds = self.activation(preds)
        for _ in range(self.num_of_layers):
            preds = self.fc6(preds)
            if activation:
                     preds = self.activation(preds)
        preds =  self.fc7(preds)
        preds = F.sigmoid(preds)
        return preds

In [46]:
class Test_Model(nn.Module):
    def __init__(self,num_of_layers=1activation=F.relu,input_shape=13,fc1_output=32,fc2_output=64,fc3_output=128,fc4_output=256,output_shape=1:
        super().__init__()
        self.num_of_layers = num_of_layers
        self.activation = activation
        self.fc1 = nn.Linear(input_shape,fc1_output)
        self.fc2 = nn.Linear(fc1_output,fc2_output)
        self.fc3 = nn.Linear(fc2_output,fc3_output)
        self.fc4 = nn.Linear(fc3_output,fc4_output)
        self.fc5 = nn.Linear(fc4_output,fc3_output)
        self.fc6 = nn.Linear(fc3_output,fc3_output)
        self.fc7 = nn.Linear(fc3_output,output_shape)
    
    def forward(self,X,activation=False):
        preds = self.fc1(X)
        if activation:
                 preds = self.activation(preds)
        preds = self.fc2(preds)
        if activation:
                 preds = self.activation(preds)
        preds = self.fc3(preds)
        if activation:
                 preds = self.activation(preds)
        preds = self.fc4(preds)
        if activation:
                 preds = self.activation(preds)
        preds = self.fc5(preds)
        if activation:
                 preds = self.activation(preds)
        for _ in range(self.num_of_layers):
            preds = self.fc6(preds)
            if activation:
                     preds = self.activation(preds)
        preds =  self.fc7(preds)
        preds = F.sigmoid(preds)
        return preds

In [47]:
class Test_Model(nn.Module):
    def __init__(self,num_of_layers=1,activation=F.relu,input_shape=13,fc1_output=32,fc2_output=64,fc3_output=128,fc4_output=256,output_shape=1:
        super().__init__()
        self.num_of_layers = num_of_layers
        self.activation = activation
        self.fc1 = nn.Linear(input_shape,fc1_output)
        self.fc2 = nn.Linear(fc1_output,fc2_output)
        self.fc3 = nn.Linear(fc2_output,fc3_output)
        self.fc4 = nn.Linear(fc3_output,fc4_output)
        self.fc5 = nn.Linear(fc4_output,fc3_output)
        self.fc6 = nn.Linear(fc3_output,fc3_output)
        self.fc7 = nn.Linear(fc3_output,output_shape)
    
    def forward(self,X,activation=False):
        preds = self.fc1(X)
        if activation:
                 preds = self.activation(preds)
        preds = self.fc2(preds)
        if activation:
                 preds = self.activation(preds)
        preds = self.fc3(preds)
        if activation:
                 preds = self.activation(preds)
        preds = self.fc4(preds)
        if activation:
                 preds = self.activation(preds)
        preds = self.fc5(preds)
        if activation:
                 preds = self.activation(preds)
        for _ in range(self.num_of_layers):
            preds = self.fc6(preds)
            if activation:
                     preds = self.activation(preds)
        preds =  self.fc7(preds)
        preds = F.sigmoid(preds)
        return preds

In [48]:
class Test_Model(nn.Module):
    def __init__(self,num_of_layers=1,activation=F.relu,input_shape=13,fc1_output=32,fc2_output=64,fc3_output=128,fc4_output=256,output_shape=1):
        super().__init__()
        self.num_of_layers = num_of_layers
        self.activation = activation
        self.fc1 = nn.Linear(input_shape,fc1_output)
        self.fc2 = nn.Linear(fc1_output,fc2_output)
        self.fc3 = nn.Linear(fc2_output,fc3_output)
        self.fc4 = nn.Linear(fc3_output,fc4_output)
        self.fc5 = nn.Linear(fc4_output,fc3_output)
        self.fc6 = nn.Linear(fc3_output,fc3_output)
        self.fc7 = nn.Linear(fc3_output,output_shape)
    
    def forward(self,X,activation=False):
        preds = self.fc1(X)
        if activation:
                 preds = self.activation(preds)
        preds = self.fc2(preds)
        if activation:
                 preds = self.activation(preds)
        preds = self.fc3(preds)
        if activation:
                 preds = self.activation(preds)
        preds = self.fc4(preds)
        if activation:
                 preds = self.activation(preds)
        preds = self.fc5(preds)
        if activation:
                 preds = self.activation(preds)
        for _ in range(self.num_of_layers):
            preds = self.fc6(preds)
            if activation:
                     preds = self.activation(preds)
        preds =  self.fc7(preds)
        preds = F.sigmoid(preds)
        return preds

In [49]:
model = Test_Model()

In [50]:
device = torch.device('cuda')

In [51]:
model = Test_Model().to(device)

In [52]:
moedl.to(device)

In [53]:
model.to(device)

Test_Model(
  (fc1): Linear(in_features=13, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=128, bias=True)
  (fc4): Linear(in_features=128, out_features=256, bias=True)
  (fc5): Linear(in_features=256, out_features=128, bias=True)
  (fc6): Linear(in_features=128, out_features=128, bias=True)
  (fc7): Linear(in_features=128, out_features=1, bias=True)
)

In [54]:
optimizer = torch.optim.Adam(model.parameters(),lr=0.25)

In [55]:
criterion = nn.BCELoss()

In [56]:
wandb.init(project=PROJECT_NAME,name='baseline')
for _ in tqdm(range(EPOCHS)):
    preds = model(X_train.float().to(device))
    preds = preds.view(len(preds),)
    preds.to(device)
    loss = criterion(preds,y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    wandb.log({'loss':loss.item(),'val_loss':get_loss(criterion,X_test,y_test,model),'accuracy':get_accuracy(X_train,y_train,model),'val_accuracy':get_accuracy(X_test,y_test,model)})
wandb.finish()

In [57]:
def get_loss(criterion,X,y,model):
    model.eval()
    with torch.no_grad():
        preds = model(X.float().to(device))
        preds = preds.view(len(preds),).to(device)
        y = y.view(len(y),).to(device)
        loss = criterion(preds,y)
    model.train()
    return loss.item()
def get_accuracy(X,y,model):
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for i in range(len(X)):
            pred = model(X[i].float().to(device))
            pred.to(device)
            if round(int(pred[0])) == round(int(y[i])):
                correct += 1
            total += 1
        if correct == 0:
            correct += 1
    model.train()
    return round(correct/total,3)

In [58]:
import wandb

In [59]:
from tqdm import tqdm

In [60]:
EPOCHS = 212
# EPOCHS = 100

In [61]:
model = Test_Model().to(device)
optimizer = torch.optim.SGD(model.parameters(),lr=0.25)
criterion = nn.L1Loss()
wandb.init(project=PROJECT_NAME,name='baseline')
for _ in tqdm(range(EPOCHS)):
    preds = model(X_train.float().to(device))
    preds = preds.view(len(preds),)
    preds.to(device)
    loss = criterion(preds,y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    wandb.log({'loss':loss.item(),'val_loss':get_loss(criterion,X_test,y_test,model),'accuracy':get_accuracy(X_train,y_train,model),'val_accuracy':get_accuracy(X_test,y_test,model)})
wandb.finish()